In [2]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.models as models
import torch.optim as optim
from torch.autograd import Variable
from math import sqrt
import matplotlib.pyplot as plt
import numpy as np
import time
import csv
from MobileNet import mobilenet_v2

EPOCHS_ARRAY = [1,5,10,15,30]
BATCH_SIZE_ARRAY = [1,2,4,8,10,16,24]
TRAIN_SIZE_ARRAY = [200,400,600,800,-1]


In [5]:
def run(epochs_arr,batchsize_arr,train_size_arr):
    EPOCHS = epochs_arr
    TRAIN_BATCH_SIZE = batchsize_arr
    TRAIN_SIZE = train_size_arr

    BATCH_SIZE = 10

    LEARNING_RATE = 0.0003
    TRAIN_DATAPATH = 'chest_xray\\train'
    VAL_DATAPATH = 'chest_xray\\val'
    TEST_DATAPATH = 'chest_xray\\test'
    TRAIN_TRANSFORM_IMG = transforms.Compose([
        transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
        transforms.RandomRotation(degrees = 15),
        transforms.ColorJitter(),
        transforms.RandomHorizontalFlip(),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])
        ])

    VAL_TRANSFORM_IMG = transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

    train_data = torchvision.datasets.ImageFolder(root=TRAIN_DATAPATH, transform=TRAIN_TRANSFORM_IMG)
    train_data_loader = torch.utils.data.DataLoader(train_data,batch_size=TRAIN_BATCH_SIZE,shuffle=True,num_workers=4)
    val_data = torchvision.datasets.ImageFolder(root=TEST_DATAPATH, transform=VAL_TRANSFORM_IMG)
    val_data_loader = torch.utils.data.DataLoader(val_data,batch_size=BATCH_SIZE,shuffle=True,num_workers=4)
    test_data = torchvision.datasets.ImageFolder(root=TEST_DATAPATH, transform=VAL_TRANSFORM_IMG)
    test_data_loader = torch.utils.data.DataLoader(test_data,batch_size=BATCH_SIZE,shuffle=True,num_workers=4)

    print_row = []
    # def imshow(img):
    #     img = img / 2 + 0.5     # unnormalize
    #     npimg = img.numpy()
    #     plt.figure(figsize=(20,20))
    #     plt.imshow(np.transpose(npimg, (1, 2, 0)))


    # dataiter = iter(train_data_loader)
    # images, labels = dataiter.next()
    # # show images
    # imshow(torchvision.utils.make_grid(images))
#     print("Number of Training Examples: ", len(train_data))
#     print("Number of Test Examples: ", len(test_data))
#     print("Number of Valid Examples: ", len(val_data))
#     print("Detected Classes are: ", train_data.class_to_idx)
    train_iter = iter(train_data_loader)
    images, labels_ = train_iter.next()
#     print("Image Shape on Batch size = {} ".format(images.size()))

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#     print(device)

    model = models.mobilenet_v2(True)
    for param in model.parameters():
        param.requires_grad = False

    model.classifier = nn.Sequential(nn.Linear(1280, 256), 
                          nn.ReLU(), 
                          nn.Dropout(0.4),
                          nn.Linear(256, 2),                   
                          nn.LogSoftmax(dim=1))

#     print(model)

    model.to(device)
    criterion = nn.NLLLoss()
    _params = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = optim.Adam(_params, lr=LEARNING_RATE, weight_decay=1e-5)

    train_loss = []
    test_loss = []
    train_accuracy = []
    test_accuracy = []

    for epoch in range(EPOCHS):
        start = time.time()
        correct = 0 
        iterations = 0
        iter_loss = 0.0

        model.train()

        for i,data in enumerate(train_data_loader,0):
            inputs, labels = data[0].to(device), data[1].to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs,labels)
            iter_loss+=loss.item()
            loss.backward()
            optimizer.step()

            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum()
            iterations += 1
            if(iterations == TRAIN_SIZE):
                break

        train_loss.append(iter_loss/iterations)
        # Record the training accuracy
        train_accuracy.append((100 * correct / len(train_data)))

        loss = 0.0
        correct = 0
        iterations = 0
        model.eval()

        for i, data in enumerate(val_data_loader):
            inputs, labels = data[0].to(device), data[1].to(device)
            outputs = model(inputs)     
            loss = criterion(outputs, labels)
            loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum()

            iterations+=1

        test_loss.append(loss/iterations)
        # Record the Testing accuracy
        test_accuracy.append((100 * correct / len(val_data)))
        stop = time.time()
#         print ('Epoch {}/{}, Training Loss: {:.3f}, Training Accuracy: {:.3f}, Validation Loss: {:.3f}, Validation Acc: {:.3f}, Time: {:.3f}s'
#                .format(epoch+1, EPOCHS, train_loss[-1], train_accuracy[-1], test_loss[-1], test_accuracy[-1], stop-start))

        row = [epoch+1,train_loss[-1],train_accuracy[-1].item(),test_loss[-1].item(),test_accuracy[-1].item(),stop-start]
        print_row.append(row)

    correct = 0
    total = 0
    classes =['NORMAL','PNEUMONIA']
    confusion_matrix = torch.zeros(2, 2)
    correct_0 = 0
    correct_1 = 0
    class_correct = list(0. for i in range(2))
    class_total = list(0. for i in range(2))
    with torch.no_grad():
        for data in test_data_loader:
            images,labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data,1)
            total += labels.size(0)
            c = (predicted == labels).squeeze()
            for i in range(4):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1
            correct+=(predicted == labels).sum().item()
            for t, p in zip(labels.view(-1), predicted.view(-1)):
                    confusion_matrix[t.long(), p.long()] += 1
    acc = 100*correct/total
#     print(correct)
#     print(total)
#     print('Testing accuracy: ' + str(acc))
#     print(confusion_matrix)

#     for i in range(2):
#         print('Accuracy of %5s : %2d %%' % (
#             classes[i], 100 * class_correct[i] / class_total[i]))
    acc0 = 100 * class_correct[0] / class_total[0]
    acc1 = 100 * class_correct[1] / class_total[1]
    print_row_end = []
    row_end = [acc,acc0,acc1]
    print_row_end.append(row_end)

    filename = "mobilenet+base_"+str(TRAIN_BATCH_SIZE)+"_"+str(TRAIN_SIZE)+"_"+str(EPOCHS)
    heading = []
    footer = []
    heading.append(["Epoch Number", 'Training Loss', 'Training Accuracy','Validation Loss','Validation Acc', 'Time' ])
    footer.append(['Testing Accuracy','Accuracy of Normal', 'Accuracy of Pneumonia'])
    with open('MobileNet/'+filename+'.csv', mode='w',newline='') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerows(heading)
        writer.writerows(print_row)
        writer.writerows(footer)
        writer.writerows(print_row_end)
    import winsound
    frequency = 2500  # Set Frequency To 2500 Hertz
    duration = 1000  # Set Duration To 1000 ms == 1 second
    winsound.Beep(frequency, duration)

#     print("PyTorch version: ", torch.__version__ )
#     print("CUDA available: ", torch.cuda.is_available())
#     print("CUDA version: ", torch.version.cuda)
    print(filename,end=" ")
    print("acc norm: "+ str(acc0),end=" ")
    print("acc pneu: "+ str(acc1))

    torch.cuda.empty_cache()

In [6]:
for i in EPOCHS_ARRAY:
    for j in BATCH_SIZE_ARRAY:
        for k in TRAIN_SIZE_ARRAY:
            run(i,j,k)
            torch.cuda.empty_cache()

mobilenet+base_1_200_1 acc norm: 0.0 acc pneu: 100.0
mobilenet+base_1_400_1 acc norm: 0.0 acc pneu: 100.0
mobilenet+base_1_600_1 acc norm: 0.0 acc pneu: 100.0
mobilenet+base_1_800_1 acc norm: 0.0 acc pneu: 100.0
mobilenet+base_1_-1_1 acc norm: 0.0 acc pneu: 100.0
mobilenet+base_2_200_1 acc norm: 64.21052631578948 acc pneu: 88.53503184713375
mobilenet+base_2_400_1 acc norm: 0.0 acc pneu: 100.0
mobilenet+base_2_600_1 acc norm: 15.306122448979592 acc pneu: 100.0
mobilenet+base_2_800_1 acc norm: 33.333333333333336 acc pneu: 98.0
mobilenet+base_2_-1_1 acc norm: 82.0 acc pneu: 90.13157894736842
mobilenet+base_4_200_1 acc norm: 17.346938775510203 acc pneu: 99.35064935064935
mobilenet+base_4_400_1 acc norm: 25.77319587628866 acc pneu: 99.35483870967742
mobilenet+base_4_600_1 acc norm: 54.76190476190476 acc pneu: 98.80952380952381
mobilenet+base_4_800_1 acc norm: 68.47826086956522 acc pneu: 96.875
mobilenet+base_4_-1_1 acc norm: 69.81132075471699 acc pneu: 97.26027397260275
mobilenet+base_8_200

mobilenet+base_2_-1_15 acc norm: 65.0 acc pneu: 97.36842105263158
mobilenet+base_4_200_15 acc norm: 73.25581395348837 acc pneu: 94.57831325301204
mobilenet+base_4_400_15 acc norm: 71.76470588235294 acc pneu: 97.0059880239521
mobilenet+base_4_600_15 acc norm: 62.88659793814433 acc pneu: 98.06451612903226
mobilenet+base_4_800_15 acc norm: 55.78947368421053 acc pneu: 97.45222929936305
mobilenet+base_4_-1_15 acc norm: 77.55102040816327 acc pneu: 97.40259740259741
mobilenet+base_8_200_15 acc norm: 71.42857142857143 acc pneu: 98.7012987012987
mobilenet+base_8_400_15 acc norm: 58.8235294117647 acc pneu: 98.66666666666667
mobilenet+base_8_600_15 acc norm: 72.94117647058823 acc pneu: 97.0059880239521
mobilenet+base_8_800_15 acc norm: 63.63636363636363 acc pneu: 98.17073170731707
mobilenet+base_8_-1_15 acc norm: 66.30434782608695 acc pneu: 99.375
mobilenet+base_10_200_15 acc norm: 50.0 acc pneu: 98.10126582278481
mobilenet+base_10_400_15 acc norm: 72.54901960784314 acc pneu: 97.33333333333333
mo